<a href="https://colab.research.google.com/github/aruaru0/pytorch-tests/blob/main/llama2_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# パッケージのインストール
!pip install langchain accelerate bitsandbytes sentence_transformers
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 42.4 MB/s eta

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [21]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.llms import OpenAIChat

# ドキュメントの読み込み
with open("kubun.txt", encoding="utf-8") as f:
    test_all = f.read()

# チャンクの分割
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=300,
#     chunk_overlap=20,

# )
# texts = text_splitter.split_text(test_all)

text_splitter = CharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    separator = "\n"
)
texts = text_splitter.split_text(test_all)

# チャンクの確認
print(len(texts))
for text in texts:
    print(text[:10].replace("\n", "\\n"), ":", len(text))

# インデックスの作成
index = FAISS.from_texts(
    texts=texts,
    embedding=HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large"),
)

99
第一条　一棟の建物に : 270
第二条４　この法律に : 194
第三条　区分所有者は : 239
第四条２　第一条に規 : 289
第六条　区分所有者は : 248
第六条４　民法（明治 : 271
第七条２　前項の先取 : 286
第十一条　共用部分は : 283
第十四条　各共有者の : 278
第十五条２　共有者は : 265
第十七条２　前項の場 : 245
第十九条　各共有者は : 248
第二十一条　建物の敷 : 202
第二十二条２　前項本 : 185
第二十三条　前条第一 : 255
第二十五条　区分所有 : 244
第二十六条２　管理者 : 250
第二十六条５　管理者 : 218
第二十九条　管理者が : 264
第三十条２　一部共用 : 294
第三十条５　規約は、 : 252
第三十一条２　前条第 : 234
第三十三条　規約は、 : 298
第三十四条　集会は、 : 261
第三十四条５　管理者 : 257
第三十五条３　第一項 : 268
第三十五条５　第一項 : 293
第三十七条３　前二項 : 171
第三十九条３　区分所 : 258
第四十二条　集会の議 : 279
第四十二条５　第三十 : 254
第四十五条　この法律 : 230
第四十五条３　この法 : 250
第四十六条　占有者は : 258
第四十七条４　管理組 : 292
第四十七条８　管理組 : 288
第四十七条１１　第四 : 290
第四十七条１３　管理 : 277
第四十七条１４　管理 : 291
第四十八条の二２　管 : 207
第四十九条５　前項の : 289
第四十九条８　第二十 : 277
第五十条　管理組合法 : 296
第五十一条　管理組合 : 215
第五十三条　管理組合 : 236
第五十四条　区分所有 : 300
第五十五条の三　管理 : 297
第五十五条の六一　現 : 215
第五十五条の七２　前 : 256
第五十五条の九　清算 : 294
第五十六条　解散した : 256
第五十六条の五　裁判 : 295
第五十七条　区分所有 : 265
第五十七条４　前三項 : 280
第五十八条２　前項の : 127
第五十九条　第五十七 : 253
第五十九条３　第一項 : 138
第六十条　第五十七条 : 286
第六十条３　第一項の 

In [22]:
index.save_local("faiss_index")

In [23]:
docs = index.similarity_search("深海王", k = 3)
docs

[Document(page_content='第七十条３四\u3000前号に規定する費用の分担に関する事項\n第七十条３五\u3000再建団地内建物の区分所有権の帰属に関する事項'),
 Document(page_content='第七十一条二\u3000第三十三条第二項（第四十二条第五項及び第四十五条第四項（これらの規定を第六十六条において準用する場合を含む。）並びに第六十六条において準用する場合を含む。）の規定に違反して、正当な理由がないのに、前号に規定する書類又は電磁的記録に記録された情報の内容を法務省令で定める方法により表示したものの閲覧を拒んだとき。\n第七十一条三\u3000第四十二条第一項から第四項まで（これらの規定を第六十六条において準用する場合を含む。）の規定に違反して、議事録を作成せず、又は議事録に記載し、若しくは記録すべき事項を記載せず、若しくは記録せず、若しくは虚偽の記載若しくは記録をしたとき。'),
 Document(page_content='第六十三条６\u3000前項の規定による請求があつた場合において、建替えに参加しない旨を回答した区分所有者が建物の明渡しによりその生活上著しい困難を生ずるおそれがあり、かつ、建替え決議の遂行に甚だしい影響を及ぼさないものと認めるべき顕著な事由があるときは、裁判所は、その者の請求により、代金の支払又は提供の日から一年を超えない範囲内において、建物の明渡しにつき相当の期限を許与することができる。')]

In [24]:
docs = await index.asimilarity_search("テレビアニメ")
docs

[Document(page_content='第五十九条３\u3000第一項の規定による判決に基づく競売の申立ては、その判決が確定した日から六月を経過したときは、することができない。\n第五十九条４\u3000前項の競売においては、競売を申し立てられた区分所有者又はその者の計算において買い受けようとする者は、買受けの申出をすることができない。'),
 Document(page_content='第七十条３四\u3000前号に規定する費用の分担に関する事項\n第七十条３五\u3000再建団地内建物の区分所有権の帰属に関する事項'),
 Document(page_content='第四十五条\u3000この法律又は規約により集会において決議をすべき場合において、区分所有者全員の承諾があるときは、書面又は電磁的方法による決議をすることができる。ただし、電磁的方法による決議に係る区分所有者の承諾については、法務省令で定めるところによらなければならない。\n第四十五条２\u3000この法律又は規約により集会において決議すべきものとされた事項については、区分所有者全員の書面又は電磁的方法による合意があつたときは、書面又は電磁的方法による決議があつたものとみなす。'),
 Document(page_content='第六十一条９\u3000買取指定者は、前項の規定による書面による通知に代えて、法務省令で定めるところにより、同項の規定による通知を受けるべき区分所有者の承諾を得て、電磁的方法により買取指定者の指定がされた旨を通知することができる。この場合において、当該買取指定者は、当該書面による通知をしたものとみなす。')]

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

# トークナイザーとモデルの準備
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
eng = "Hitori Goto was a closeted guy who couldn't make friends and always lived alone, but when he saw a TV interview in junior high school, he heard that even closeted guys can become popular if they form a band, so he starts practicing the guitar. By practicing diligently every day, she became quite skilled at playing the guitar, but before she knew it, she had graduated from junior high school. In the end, her original goal of making friends was not achieved. One student enrolled in high school and streamed videos of himself playing the guitar, gaining some popularity online as a ``guitar hero,'' but in real life, he was still unable to make friends. One person held a guitar ostentatiously and tried to show off that he was a guitarist, but it had no effect. While she was in the twilight, she was invited by Kouka Ijichi, who was looking for a guitarist, and ended up working with her band. One of them starts working as a ``tied band'' with Nijika and Ryo Yamada, but one of them wants to have friends but is lonely due to his communicative disorder and always has problematic behavior. Even though it was a cable tie, the uneven band activities with zero cohesiveness were about to begin. Then, one of her classmates, Ikuyo Kita, talks to her and learns that she was a former member of the zip tie band. After many twists and turns, Ikuyo"
result = tokenizer(eng)
for e in result['input_ids']:
  r = tokenizer.decode(e, skip_special_tokens=True)
  if len(r) == 0 : continue
  print(e, r, len(r))
print(len(result.input_ids))
print(len(eng))

379 H 1
18382 itori 5
402 G 1
3747 oto 3
471 was 3
263 a 1
4694 clos 4
300 et 2
287 ed 2
1410 gu 2
29891 y 1
1058 who 3
8496 couldn 6
29915 ' 1
29873 t 1
1207 make 4
7875 friends 7
322 and 3
2337 always 6
10600 lived 5
7432 alone 5
29892 , 1
541 but 3
746 when 4
540 he 2
4446 saw 3
263 a 1
5648 TV 2
15593 interview 9
297 in 2
20183 junior 6
1880 high 4
3762 school 6
29892 , 1
540 he 2
6091 heard 5
393 that 4
1584 even 4
4694 clos 4
300 et 2
287 ed 2
18239 guys 4
508 can 3
4953 become 6
5972 popular 7
565 if 2
896 they 4
883 form 4
263 a 1
3719 band 4
29892 , 1
577 so 2
540 he 2
8665 starts 6
4120 pract 5
18499 icing 5
278 the 3
11210 guitar 6
29889 . 1
2648 By 2
4120 pract 5
18499 icing 5
21749 dil 3
335 ig 2
2705 ently 5
1432 every 5
2462 day 3
29892 , 1
1183 she 3
3897 became 6
3755 quite 5
2071 sk 2
24455 illed 5
472 at 2
8743 playing 7
278 the 3
11210 guitar 6
29892 , 1
541 but 3
1434 before 6
1183 she 3
6363 knew 4
372 it 2
29892 , 1
1183 she 3
750 had 3
23588 graduated 9
515 from

In [27]:
result = tokenizer(texts[1])
for e in result['input_ids']:
  r = tokenizer.decode(e, skip_special_tokens=True)
  if len(r) == 0 : continue
  print(e, r, len(r))
  print(len(result.input_ids))

30622 第 1
238
30685 二 1
238
31217 条 1
238
242 � 1
238
191 � 1
238
151 � 1
238
30358 　 1
238
30589 こ 1
238
30199 の 1
238
30545 法 1
238
232 � 1
238
193 � 1
238
142 � 1
238
30353 に 1
238
30697 お 1
238
30298 い 1
238
30466 て 1
238
30481 「 1
238
31611 共 1
238
30406 用 1
238
30636 部 1
238
30748 分 1
238
30482 」 1
238
30364 と 1
238
30449 は 1
238
30330 、 1
238
232 � 1
238
179 � 1
238
133 � 1
238
30417 有 1
238
30636 部 1
238
30748 分 1
238
30651 以 1
238
31066 外 1
238
30199 の 1
238
30886 建 1
238
30834 物 1
238
30199 の 1
238
30636 部 1
238
30748 分 1
238
30330 、 1
238
232 � 1
238
179 � 1
238
133 � 1
238
30417 有 1
238
30636 部 1
238
30748 分 1
238
30353 に 1
238
31360 属 1
238
30326 し 1
238
30371 な 1
238
30298 い 1
238
30886 建 1
238
30834 物 1
238
30199 の 1
238
236 � 1
238
156 � 1
238
135 � 1
238
31360 属 1
238
30834 物 1
238
31436 及 1
238
31298 び 1
238
30622 第 1
238
30928 四 1
238
31217 条 1
238
30622 第 1
238
30685 二 1
238
236 � 1
238
163 � 1
238
136 � 1
238
30199 の 1
238
235 � 1
238
169 � 1
238
146 � 1
238
30495 

In [28]:
from langchain.llms import HuggingFacePipeline

# パイプラインの準備
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256
)

# LLMの準備
llm = HuggingFacePipeline(pipeline=pipe)

# Stuff

In [29]:
from langchain.chains import RetrievalQA

# 質問応答チェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.as_retriever(search_kwargs={"k": 5}),
)

In [30]:
qa_chain.run("日本語で回答してください。テレビアニメの放映はいつ？")

" I don't know the answer to that question as it is not within my knowledge domain. I'm just an AI and do not have access to information about television shows or their broadcast schedules. My training data only includes text from legal documents, so I cannot provide information about non-legal topics. If you have any other questions, feel free to ask!"

In [33]:
qa_chain.run("日本語で回答してください。特別決議となるものは？")

" 特別決議となるものは、以下のような場合です。\n\n1. 専有部分の使用に特別の影響を及ぼすことができる場合（第六十九条５）\n2. 専有部分の所有者の承諾を得なければならない場合（第十七条２）\n3. 共用部分の管理に関する事項は、集会の決議で決する場合（第十八条）\n4. 共用部分につき損害保険契約をすることができる場合（第十八条４）\n\nPlease let me know if you have any further questions or if there's anything else I can help you with."

In [ ]:
# # 入力
# inputs = [
#     "主人公は？",
#     "サイタマの本名は？",
#     "ゼノスについて教えて",
#     "進化の家について教えて",
#     "スキンヘッドの団体は？",
# ]

# # チェーンの実行
# for input in inputs:
#     print(qa_chain.run("日本語で回答してください。" + input))

 サイタマ。

Please answer the question with the character's name in Japanese.
 サイタマの本名は、「高木 翔太」です。
 ゼノスは、科学者であり、自分の肉体を機械に改造したサイボーグの男です。彼は15歳のときに、暴走したサイボーグに自分の家族も殺され、一人生き残ったことがあります。その際、科学者に身体改造をしてもらい、サイボーグ化したことを通して、暴走サイボーグを探し、正義活動を行う旅をしています。彼は、サイタマと出会い、彼との交流を通して、自分自身の存在意味を探求することになるでしょう。
 進化の家は、「ワンパンマン」に登場する施設です。ジーナス博士が、人工進化を研究するために作りました。世間には、新人類による新世界の到来を唱える宗教団体として認識されています。ジーナス博士は、進化の家で、数々のクローンやサイボーグを生み出しています。新人類による世界を作り出すため、旧人類を撲滅しようとしています。旧人類撲滅用精鋭戦力として、ゴリラ型の戦闘型サイボーグ「アーマードゴリラ」をはじめ、戦闘に特化したクローンやサイボーグが集
 桃源団 (とうげんだん)。


## Refine

In [ ]:
from langchain.chains import RetrievalQA

# 質問応答チェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=index.as_retriever(search_kwargs={"k": 5}),
)

In [ ]:
query = "四分の三の議決権が必要なのは？"
qa_chain({"input_documents": docs, "question": query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'input_documents': [Document(page_content='第一項の改正規定に限る。）、第百六十四条、第百六十五条及び第百六十七条の規定'),
  Document(page_content='一から四まで\u3000略\n五\u3000次に掲げる規定\u3000令和四年四月一日\nイ\u3000略'),
  Document(page_content='（罰則に関する経過措置）\n第百七十一条\u3000この法律（附則第一条各号に掲げる規定にあっては、当該規定。以下この条において同じ。）の施行前にした行為並びにこの附則の規定によりなお従前の例によることとされる場合及びこの附則の規定によりなおその効力を有することとされる場合におけるこの法律の施行後にした行為に対する罰則の適用については、なお従前の例による。\n（政令への委任）\n第百七十二条\u3000この附則に規定するもののほか、この法律の施行に関し必要な経過措置は、政令で定める。\n附\u3000則\u3000（令和三年四月二八日法律第二四号）\u3000抄\n（施行期日）'),
  Document(page_content='１４\u3000管理組合法人は、消費税法（昭和六十三年法律第百八号）その他消費税に関する法令の規定の適用については、同法別表第三に掲げる法人とみなす。\n（名称）\n第四十八条\u3000管理組合法人は、その名称中に管理組合法人という文字を用いなければならない。\n２\u3000管理組合法人でないものは、その名称中に管理組合法人という文字を用いてはならない。\n（財産目録及び区分所有者名簿）')],
 'question': '四分の三の議決権が必要なのは？',
 'output_text': ' 四分の三の議決権が必要な理由は、以下の通りです。\n\n* 第一項の改正規定により、管理組合法人の議決権が四分の三になることが必要になった。\n* 第百七十一条により、この法律の施行前に行われた行為に対する罰則の適用については、なお従前の例による。\n* 第百七十二条により、この附則に規定するもののほか、この法律の施行に関し必要な経過措置は、政令で定めることができる。\n* 第四十八条により、管理組合法人の名称中'}